# Performing EDA on Venmo data

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import pymongo
import json
import datetime
import pickle

Data has to be exported from a Mongo DB

In [ ]:
# Instantiate a MongoClient and inspect the database names
mc = pymongo.MongoClient()
mc.list_database_names()

In [ ]:
# Create a db from the test database in MongoClient
mydb = mc['test']

In [ ]:
# Accessing the venmo collection in the test database
venmo = mydb['venmo']

In [ ]:
# Count all transactions in the venmo data
venmo.count()

In [ ]:
# Inspect the first transaction
venmo.find_one()

In [ ]:
venmo_transactions = venmo.find({})

In [ ]:
first_700k = []
counter = 0
for transaction in venmo_transactions:
    if counter <= 700000:
        first_700k.append(transaction)
        counter += 1
    else:
        break

In [ ]:
with open('first_10%_transactions.pkl', 'wb') as f:
    pickle.dump(first_700k, f)

In [ ]:
#jul_midaug_2018 = []
#for transaction in venmo_transactions:
#    aug_end = datetime.datetime(2018, 8, 15, 11, 59, 59)
#    if transaction['date_created'] <= aug_end:
#        jul_midaug_2018.append(transaction)
#    else:
#        break

In [ ]:
first.keys()

In [ ]:
first['payment']['actor']

In [ ]:
first['payment']['target']['user']

In [ ]:
type(first['payment']['actor'])

In [ ]:
first['payment']['actor'].items()

In [ ]:
transactions = [transaction for transaction in venmo]

In [ ]:
user = {}
user_ids = []
for key, val in venmo['payment']['actor'].items():
    if key == 'id':
        if val in user_ids:
            continue
        else:
            user_ids.append(val)
    user[key] = val

In [ ]:
user

In [ ]:
users = []
user_ids = []
for key, val in first['payment']['actor'].items():
    users
    
    user_id = transaction[0]
    user_ids.append(user_id)
    if user_id in user_ids:
        continue
    else:
        users.append(transaction['payment']['actor'])

In [ ]:
# extracting the information of each payer (actor) and each payee (user)
first_actor = first

In [ ]:
first_df